In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:

df = pd.read_csv('preprocessed_data.csv')

In [3]:
df = df.dropna(subset=['cleaned_headline'])

In [4]:

label_encoder = LabelEncoder()
df['category_encoded'] = label_encoder.fit_transform(df['category'])

In [5]:

X = df['cleaned_headline']
y = df['category_encoded']
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.25, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.6, random_state=42)

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2023-09-21 13:10:34.659754: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
max_words = 10000  
max_seq_length = 100  

In [8]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

In [9]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [10]:
X_train_pad = pad_sequences(X_train_seq, maxlen=max_seq_length, padding='post')
X_val_pad = pad_sequences(X_val_seq, maxlen=max_seq_length, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_seq_length, padding='post')

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, LSTM, Dense

In [12]:
embedding_dim = 100  
num_classes = len(label_encoder.classes_)

In [13]:
from tensorflow.keras.layers import Reshape

In [14]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_seq_length))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(GlobalMaxPooling1D())

In [15]:

model.add(Reshape((1, 128)))

In [16]:
model.add(LSTM(128))
model.add(Dense(num_classes, activation='softmax'))

In [17]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
batch_size = 64  
epochs = 10  

history = model.fit(X_train_pad, y_train, validation_data=(X_val_pad, y_val), epochs=epochs, batch_size=batch_size)


Epoch 1/10
2454/2454 [==============================] - 189s 75ms/step - loss: 2.1624 - accuracy: 0.4433 - val_loss: 1.7140 - val_accuracy: 0.5374
Epoch 2/10
2454/2454 [==============================] - 211s 86ms/step - loss: 1.5276 - accuracy: 0.5796 - val_loss: 1.5889 - val_accuracy: 0.5633
Epoch 3/10
2454/2454 [==============================] - 182s 74ms/step - loss: 1.2639 - accuracy: 0.6396 - val_loss: 1.6115 - val_accuracy: 0.5581
Epoch 4/10
2454/2454 [==============================] - 184s 75ms/step - loss: 1.0260 - accuracy: 0.7037 - val_loss: 1.7176 - val_accuracy: 0.5556
Epoch 5/10
2454/2454 [==============================] - 194s 79ms/step - loss: 0.8064 - accuracy: 0.7673 - val_loss: 1.9045 - val_accuracy: 0.5415
Epoch 6/10
2454/2454 [==============================] - 185s 75ms/step - loss: 0.6243 - accuracy: 0.8204 - val_loss: 2.1590 - val_accuracy: 0.5257
Epoch 7/10
2454/2454 [==============================] - 188s 77ms/step - loss: 0.4878 - accuracy: 0.8611 - val_loss: 2

In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [20]:
y_pred = model.predict(X_test_pad)
y_pred_classes = y_pred.argmax(axis=-1)

accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes, average='weighted')
recall = recall_score(y_test, y_pred_classes, average='weighted')
f1 = f1_score(y_test, y_pred_classes, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

982/982 [==============================] - 11s 10ms/step
Accuracy: 0.4992518068069662
Precision: 0.4904970360802164
Recall: 0.4992518068069662
F1 Score: 0.49276402873404207
